In [12]:
import numpy as np
import utils.optlib as opt
import plotly.graph_objects as go

In [13]:
# setting the underlying
stock = opt.Underlying(price=65,
                       volatility=0.3,
                       interest=0.1)

# creating the European Put option
EuPut = opt.Option(underlying=stock,
                   style="European",
                   call=False,
                   strike=50)

# creating the grid and adding option's parameters
grid = opt.Grid(xSteps=300,
                tSteps=200,
                xLeft=-3.0,
                xRight=3.0)

grid.addOption(EuPut)
print("Curant number", grid.lamda)
# setting boundary conditions

Curant number 0.5625


In [14]:
net = grid.net.copy()
q = grid.q
n = grid.xSteps
c = grid.lamda
# creating matrices for schemes
A = np.diag(np.ones(n+1) * (1 + c))
B = np.diag(np.ones(n+1) * (1 - c))
for i in range(1, n + 1):
    A[i, i-1] = -c/2
    A[i-1, i] = -c/2
    B[i, i-1] = c/2
    B[i-1, i] = c/2

In [15]:
# setting initial condition (without boundaries)
for i in np.arange(0, grid.xSteps):
    net[i, 0] = opt.g_func(q, grid.tGrid[0], grid.xGrid[i])

# crank-nickolson scheme
for i in np.arange(0, grid.tSteps):
    # explicit step
    f = np.dot(B, net[:, i].copy())
    f[0] = f[0] + 0.5 * c * (opt.g_func(q, grid.tGrid[i], grid.xGrid[0]) + opt.g_func(q, grid.tGrid[i+1], grid.xGrid[0]))
    # implicit step
    solution = opt.scalar_walk(A, f)
    net[:, i + 1] = solution

In [16]:
grid.net = net
grid.toNormal(obj_mutation=True)
grid.plot(0, cut=True)